# Detalles de TTree

## Estructura y terminología de archivos ROOT

Un archivo ROOT ([ROOT TFile](https://root.cern.ch/doc/master/classTFile.html), [uproot.ReadOnlyFile](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyFile.html)) es como un pequeño sistema de archivos que contiene directorios anidados ([ROOT TDirectory](https://root.cern.ch/doc/master/classTDirectory.html), [uproot.ReadOnlyDirectory](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html)). En Uproot, los directorios anidados se presentan como diccionarios anidados.

Cualquier instancia de clase ([ROOT TObject](https://root.cern.ch/doc/master/classTObject.html), [uproot.Model](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html)) puede almacenarse en un directorio, incluidos tipos como histogramas (por ejemplo, [ROOT TH1](https://root.cern.ch/doc/master/classTH1.html), [uproot.behaviors.TH1.TH1](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TH1.TH1.html)).

Una de estas clases, TTree ([ROOT TTree](https://root.cern.ch/doc/master/classTTree.html), [uproot.TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html)), es una puerta de entrada a conjuntos de datos grandes. Un TTree es algo parecido a un DataFrame de Pandas en el sentido de que representa una tabla de datos. Las columnas se llaman TBranches ([ROOT TBranch](https://root.cern.ch/doc/master/classTBranch.html), [uproot.TBranch](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html)), que pueden ser anidadas (a diferencia de Pandas), y los datos pueden tener cualquier tipo de C++ (a diferencia de Pandas, que puede almacenar cualquier tipo de Python).

Un TTree a menudo es demasiado grande para caber en la memoria, y a veces (raramente) incluso una sola TBranch es demasiado grande para caber en la memoria. Cada TBranch se divide en TBaskets ([ROOT TBasket](https://root.cern/doc/master/classTBasket.html), [uproot.models.TBasket.Model_TBasket](https://uproot.readthedocs.io/en/latest/uproot.models.TBasket.Model_TBasket.html)), que son "lotes" de datos. (Estos son los mismos lotes que cada llamada a `extend` escribe en la lección anterior.) Los TBaskets son la unidad más pequeña que se puede leer de un TTree: si deseas leer la primera entrada, debes leer el primer TBasket.

![terminology](fig/terminology.png)

Como analista de datos, probablemente te ocuparás de los TTrees y TBranches de manera directa, pero solo de los TBaskets cuando surjan problemas de eficiencia. Los archivos con TBaskets grandes pueden requerir mucha memoria para leer; los archivos con TBaskets pequeños serán más lentos de leer (en ROOT también, pero especialmente en Uproot). Los TBaskets del tamaño de megabytes suelen ser ideales.

## Ejemplos con un TTree grande

[Este archivo](http://opendata.web.cern.ch/record/12341) tiene 2.1 GB y está alojado en el Portal de Datos Abiertos de CERN.

In [ ]:
import uproot

archivo = uproot.open(
    "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
)
archivo.classnames()

```{admonition} ¿Por qué el ;74 y ;75?
Tal vez te hayas preguntado sobre los números después de los puntos y comas. Estos son los "números de ciclo" de ROOT, que permiten distinguir objetos con el mismo nombre. Se utilizan cuando un objeto necesita sobrescribirse a medida que crece sin perder la última copia válida de ese objeto, de modo que un archivo ROOT pueda leerse incluso si el proceso de escritura falló a mitad de camino.

En este caso, la última versión de este TTree es el número 75, y el número 74 es la penúltima.

Si no especificas números de ciclo, Uproot seleccionará el último por ti, lo cual es casi siempre lo que deseas. (En otras palabras, puedes ignorarlos.)
```

Simplemente solicitando el objeto [uproot.TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) e imprimiendolo *no* lee todo el conjunto de datos.

In [ ]:
tree = archivo["Events"]
tree.show()

### Leyendo una parte de un TTree

En la lección anterior, aprendimos que la forma más directa de leer una TBranch es llamando a [uproot.TBranch.array](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html#array).

In [ ]:
tree["nMuon"].array()

Sin embargo, toma mucho tiempo porque se tiene que enviar una gran cantidad de datos a través de la red.

Para limitar la cantidad de datos leídos, establece `entry_start` y `entry_stop` en el rango que desees. El `entry_start` es inclusivo, `entry_stop` es exclusivo, y la primera entrada se indexaría por `0`, al igual que los cortes en una interfaz de arreglo (primera lección). Uproot solo lee la cantidad de TBaskets necesarias para proporcionar estas entradas.

In [ ]:
tree["nMuon"].array(entry_start=1_000, entry_stop=2_000)

Estos son los bloques de construcción de un lector de datos en paralelo: cada uno es responsable de un fragmento diferente. (Consulta también [uproot.TTree.num_entries_for](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#num-entries-for) y [uproot.TTree.common_entry_offsets](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#common-entry-offsets), que se pueden usar para elegir `entry_start`/`entry_stop` de manera óptima.)

### Leer múltiples TBranches a la vez

Supongamos que sabes que necesitarás todos los TBranches de muones. Pedirlos en una sola solicitud es más eficiente que solicitarlos individualmente, porque el servidor puede estar trabajando en la lectura de los TBaskets posteriores del disco mientras los TBaskets anteriores se envían a ti a través de la red. Mientras que un TBranch tiene un método `array`, el TTree tiene un método `arrays` (en plural) para obtener múltiples arreglos.

In [ ]:
muones = tree.arrays(
    ["Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass", "Muon_charge"], entry_stop=1_000
)
muones

Ahora, los cinco TBranches están en la salida, `muones`, que es un Awkward Array. Un Awkward Array de múltiples TBranches tiene una interfaz similar a un diccionario, por lo que podemos obtener cada variable de él por

In [ ]:
muones["Muon_pt"]
muones["Muon_eta"]
muones["Muon_phi"]  # etc.

````{admonition} ¡Cuidado! ¡Es tree.arrays lo que realmente lee los datos!
Si no tienes cuidado con la llamada a [uproot.TTree.arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#arrays), podrías terminar esperando mucho tiempo por datos que no necesitas o podrías quedarte sin memoria. Leer todo con

```python
todo = tree.arrays()
```
y luego seleccionar los arrays que deseas generalmente no es una buena idea. Al menos, establece un `entry_stop`.
````

### Selección de TBranches por nombre

Supongamos que tienes muchos TBranches de muones y no quieres enumerarlos todos. Tanto [uproot.TTree.keys](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#keys) como [uproot.TTree.arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#arrays) aceptan un argumento `filter_name` que puede seleccionarlos de varias maneras (consulta la documentación). En particular, es recomendable usar primero `keys` para saber qué ramas coinciden con tu filtro, seguido de `arrays` para leerlas realmente.

In [ ]:
tree.keys(filter_name="Muon_*")

In [ ]:
tree.arrays(filter_name="Muon_*", entry_stop=1_000)

(También hay `filter_typename` y `filter_branch` para más opciones.)

## Escalando, haciendo un gráfico

La mejor manera de entender lo que estás haciendo es experimentar con conjuntos de datos pequeños y luego escalarlos. Aquí, tomamos 1000 eventos y calculamos las masas de dimuones.

In [ ]:
muones = tree.arrays(entry_stop=1_000)
corte = muones["nMuon"] == 2

pt0 = muones["Muon_pt", corte, 0]
pt1 = muones["Muon_pt", corte, 1]
eta0 = muones["Muon_eta", corte, 0]
eta1 = muones["Muon_eta", corte, 1]
phi0 = muones["Muon_phi", corte, 0]
phi1 = muones["Muon_phi", corte, 1]

import numpy as np

masa = np.sqrt(2 * pt0 * pt1 * (np.cosh(eta0 - eta1) - np.cos(phi0 - phi1)))

import hist

histmasa = hist.Hist(hist.axis.Regular(120, 0, 120, label="masa [GeV]"))
histmasa.fill(masa)
histmasa.plot()

Eso funcionó (hay un pico en Z). Ahora, para hacer esto en todo el archivo, debemos tener más cuidado con lo que estamos leyendo.

In [ ]:
tree.keys(filter_name=["nMuon", "/Muon_(pt|eta|phi)/"])

y acumular datos gradualmente con [uproot.TTree.iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#iterate). Esto maneja `entry_start`/`entry_stop` en un bucle.

In [ ]:
histmasa = hist.Hist(hist.axis.Regular(120, 0, 120, label="masa [GeV]"))

for muones in tree.iterate(filter_name=["nMuon", "/Muon_(pt|eta|phi)/"]):
    cut = muones["nMuon"] == 2
    pt0 = muones["Muon_pt", cut, 0]
    pt1 = muones["Muon_pt", cut, 1]
    eta0 = muones["Muon_eta", cut, 0]
    eta1 = muones["Muon_eta", cut, 1]
    phi0 = muones["Muon_phi", cut, 0]
    phi1 = muones["Muon_phi", cut, 1]
    masa = np.sqrt(2 * pt0 * pt1 * (np.cosh(eta0 - eta1) - np.cos(phi0 - phi1)))
    histmasa.fill(masa)
    print(histmasa.sum() / tree.num_entries)

histmasa.plot()

### Obtener datos en NumPy o Pandas

En todos los ejemplos anteriores, los métodos `array`, `arrays` e `iterate` devuelven arreglos Awkward. La biblioteca Awkward es útil para este tipo de datos (arreglos irregulares: más en la próxima lección), pero es posible que estés trabajando con bibliotecas que solo reconocen arreglos de NumPy o DataFrames de Pandas.

Utiliza `library="np"` o `library="pd"` para obtener NumPy o Pandas, respectivamente.

In [ ]:
tree["nMuon"].array(library="np", entry_stop=10_000)

In [ ]:
tree.arrays(library="np", entry_stop=10_000)

In [ ]:
tree.arrays(library="pd", entry_stop=10_000)

NumPy es excelente para datos no irregulares como la rama `"nMuon"`, pero tiene que representar un número desconocido de muones por evento como un arreglo de arreglos de NumPy (es decir, objetos de Python).

Pandas se puede hacer representar múltiples partículas por evento colocando esta estructura en un [pd.MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html), pero no cuando el DataFrame contiene más de un tipo de partícula (por ejemplo, muones *y* electrones). Usa DataFrames separados para estos casos. Si ayuda, ten en cuenta que hay otra ruta a DataFrames: leyendo los datos como un Arreglo Awkward y llamando a [ak.to_pandas](https://awkward-array.readthedocs.io/en/latest/_auto/ak.to_pandas.html) sobre él. (Algunos métodos usan más memoria que otros, y he encontrado que Pandas es inusualmente intensivo en memoria.)

O usa Arreglos Awkward (próxima lección).